In [ ]:
from pathlib import Path
import numpy as np

import sys
sys.path.append("..")
import ICE.utils

sys.path.append("/share/home/francesco/concept_composers/DeepComposerClassification/")
sys.path.append("/share/home/francesco/concept_composers/")

utils = ICE.utils.img_utils()

In [ ]:
mozart_path = Path("/share/cp/projects/concept_composers/experiments/kim2020/npy/composer12")

for folder in mozart_path.iterdir():
    if len(list(folder.iterdir())) != 1:
        print(folder.name)

In [ ]:
for folder in mozart_path.iterdir():
    for piece in folder.iterdir():
        out_path = Path("/share/home/francesco/InvertibleCE/music_experiments/dataset_check",folder.name + piece.name + ".mid")
        pr = np.transpose(np.load(piece), (0, 2, 1))
        utils.pianoroll2midi(pr, out_path, samples_per_second=20)


# Try with different fs

In [ ]:
from data_handling.io import midi_to_3dpianoroll
from DeepComposerClassification.generator import Generator

In [ ]:
generator = Generator()

def midi2pianoroll(midi_path, out_npy_path, frame_length):
    x = open_midi(str(midi_path))
    x = generator.generate_segment(x, frame_length = frame_length)
    x = np.array(x)
    np.save(str(out_npy_path), x)

In [ ]:
midi_path = Path("/share/cp/projects/concept_composers/maestro-v2.0.0/2015/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_01_R1_2015_wav--3.midi")
npy_path = Path("/share/home/francesco/InvertibleCE/music_experiments/dataset_check/pianoroll_ex.npy")
recreated_midi_path = ("/share/home/francesco/InvertibleCE/music_experiments/dataset_check/new_midi.mid")

midi2pianoroll(midi_path,npy_path, frame_length = 0.05)

In [ ]:
pr = np.transpose(np.load(npy_path), (0, 2, 1))
utils.pianoroll2midi(pr, str(recreated_midi_path), samples_per_second=int(1/0.001))

# Try only with partitura

In [ ]:
import partitura
from data_handling.io import pianoroll2midi, midi_to_3dpianoroll
midi_path = Path("/share/cp/projects/concept_composers/maestro-v2.0.0/2015/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_03_R1_2015_wav--4.midi")
npy_path = Path("/share/home/francesco/InvertibleCE/music_experiments/dataset_check/pianoroll_ex.npy")
recreated_midi_path = ("/share/home/francesco/InvertibleCE/music_experiments/dataset_check/new_midi.mid")

In [ ]:
# def midi_to_3dpianoroll(midi_path, out_npy_path, frame_length, piano_range):
#     part = partitura.load_performance_midi(midi_path)
#     matrix1 = partitura.utils.compute_pianoroll(part,time_unit = "sec", time_div= int(1/frame_length), onset_only=True, piano_range=piano_range).toarray()
#     matrix1[matrix1>0] = 1
#     matrix2 = partitura.utils.compute_pianoroll(part,time_unit = "sec", time_div= int(1/frame_length), onset_only=False, piano_range=piano_range).toarray()
#     pr3d = np.stack([matrix1,matrix2])
#     pr3d = np.transpose(pr3d, (0,2,1))
#     np.save(str(out_npy_path), pr3d)

In [ ]:
midi_to_3dpianoroll(midi_path,npy_path, frame_length = 0.05, piano_range = True)
pr = np.transpose(np.load(npy_path), (0, 2, 1))
print(pr.shape)
pianoroll2midi(pr, str(recreated_midi_path), samples_per_second=int(1/0.05))

# Checking liszt not imported

In [1]:
import partitura

In [2]:
mid = partitura.load_performance_midi("/share/cp/projects/concept_composers/maestro-v2.0.0/2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--3.midi")
# mid = partitura.load_performance_midi("/share/cp/projects/concept_composers/ConceptDataset/midi/alberti_bass/track17.mid")

In [3]:
mid.sustain_pedal_threshold = 127 # don't take pedal into consideration
partitura.utils.compute_pianoroll(
    mid,
    onset_only = True)
# matrix1[matrix1 > 0] = 1  # we don't want velocity here
# matrix2 = partitura.utils.compute_pianoroll(
#     mid,
#     time_unit="sec",
#     time_div=int(1 / frame_length),
#     onset_only=False,
#     piano_range=piano_range,
# ).toarray()
# pr3d = np.stack([matrix1, matrix2])
# pr3d = np.transpose(pr3d, (0, 2, 1))

ValueError: column index exceeds matrix dimensions

In [ ]:
# partitura.utils.compute_pianoroll(mid,time_unit = "sec", time_div= int(1/0.05), piano_range=True ).toarray()
matrix1 = partitura.utils.compute_pianoroll(mid, onset_only=True).toarray()
matrix1[matrix1>0] = 1

In [ ]:
generator = Generator()
generator.generate_segment(mid)

In [ ]:
import pretty_midi as pm
from operator import attrgetter

pmidi = pm.PrettyMIDI("/share/cp/projects/concept_composers/ConceptDataset/midi/alberti_bass/track17.mid")
print(len(pmidi.instruments[0].notes))

# for n in pmidi.instruments[0].notes.sort(key = attrgetter('start')):
    # print(n)

counter= 0
for n in pmidi.instruments[0].notes:
    print(n.pitch, n.start, n.velocity)
    counter +=1
    if n.start is None: 
        print("aaa")


print(counter)

In [ ]:
pt_out= list(mid.note_array[["pitch","onset_sec","velocity"]])
pm_out = [(n.pitch, n.start,n.end, n.velocity) for n in pmidi.instruments[0].notes]
pm_out.sort(key=lambda x: x[1])
pm_out

In [ ]:
len(pt_out)
len(pm_out)

In [ ]:
for i in range(len(pm_out)-1):
    print(pm_out[i], pt_out[i])
